In [1]:
import causalml
import math
import matplotlib.pyplot as plt
import numpy as np
from causalml.inference.meta import BaseRRegressor
from causalml.inference.meta import LRSRegressor
from causalml.propensity import ElasticNetPropensityModel
from matplotlib import pyplot as plt
from numpy import mean
from numpy import std
from opossum import UserInterface
from sklearn.datasets import make_regression
from sklearn.datasets import make_spd_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.


In [2]:
## create datasets in an array?
## for each thing in the data set, take each array and split into 1/3. two new arrays : Test and StackingTrain. 
# dataSetArray...
N = 3000
k = 10
seed= 5
u = UserInterface(N, k, seed=seed, categorical_covariates = None)

X={}
assignment={}
y={}
treatment={}
propensityScores={}


def splitArrays_helper(l):
    return np.array_split(l, 3)
def addDatasets_depreciated(y_gen, X_gen, assignment_gen, treatment_gen):
    X.append(splitArrays_helper(X_gen))
    y.append(splitArrays_helper(y_gen))
    assignment.append(splitArrays_helper(assignment_gen))
    treatment.append(splitArrays_helper(treatment_gen))
    

def splitArrays(l):
    train, stack, test = np.array_split(l, 3)
    return {'train' : train, 'stack' : stack, 'test': test}
    
def addDatasets(data_gen_name, y_gen, X_gen, assignment_gen, treatment_gen):
    y[data_gen_name] = splitArrays(y_gen)
    X[data_gen_name] = splitArrays(X_gen)
    assignment[data_gen_name] = splitArrays(assignment_gen)
    treatment[data_gen_name] = splitArrays(treatment_gen)


####IMPORTANT: after generating the data, we wish to split the dataset into three parts:
## one for training the R learner
## one for fitting the OLS stacking model
## and one for testing.

## after generating a dataset, we add the dataset to a dict of datasets. each key-value pair is a data generating function
## with three sub-dicts: one train, one stacking, one test array. 
## accessing the training dataset of the second data generating funciton is then: X['setup_B']['train']
## accessing the treatment vector (true treatment effect) of the first data set, testing data: treatment['setup_A']['test']

In [3]:
##############################################################################################
## setupA => difficult nuisance com-ponents and an easy treatment effect function
##############################################################################################
u.generate_treatment(random_assignment = False, 
                     assignment_prob = 'low', 
                     constant_pos = False, 
                     constant_neg = False,
                     heterogeneous_pos = True, 
                     heterogeneous_neg = False, 
                     no_treatment = False, 
                     discrete_heterogeneous = False,
                     treatment_option_weights = None, 
                     intensity = 10)

y_A, X_A, assignment_A, treatment_A = u.output_data(binary=False, 
                                               x_y_relation = 'nonlinear_interaction')

addDatasets("setup_A", y_A, X_A, assignment_A, treatment_A)

In [4]:
##############################################################################################
## setupB => randomized trial
##############################################################################################
u.generate_treatment(random_assignment = True, 
                     assignment_prob = 0.5, 
                     treatment_option_weights = [0.0, 0.0, 0.4, 0.6, 0.0, 0.0],
                     intensity = 5)

y_B, X_B, assignment_B, treatment_B = u.output_data(binary=False, x_y_relation = 'linear_simple')

addDatasets("setup_B", y_B, X_B, assignment_B, treatment_B)

In [5]:
##############################################################################################
## setupC => easy propensity score and difficult baseline
##############################################################################################
u.generate_treatment(random_assignment = False, 
                     assignment_prob = 'low', 
                     constant_pos = True, 
                     constant_neg = False,
                     heterogeneous_pos = False, 
                     heterogeneous_neg = False, 
                     no_treatment = False, 
                     discrete_heterogeneous = False,
                     treatment_option_weights = None, 
                     intensity = 10)

y_C, X_C, assignment_C, treatment_C = u.output_data(binary=False, 
                                               x_y_relation = 'nonlinear_interaction')

addDatasets("setup_C", y_C, X_C, assignment_C, treatment_C)
##############################################################################################
## setupD => unrelated treatment and control arms???
##############################################################################################


In [6]:
## Get propensity scores using CausalML package. (scores for each dataset and save them in an array)
propensityScores_train = []

for x in range(3):
    pm = ElasticNetPropensityModel(n_fold=5, random_state=42)
    estimatedpropensityscores = pm.fit_predict(X['setup_A']['train'], assignment['setup_A']['train'])
    propensityScores_train.append(estimatedpropensityscores)
    
    
#### INFO : the R learner (and others) need propensity scores. there are many ways to compute propensity scores. 
#### we can do them on the fly when we call the BaseRRegressor, and estimmate_ate .. default is actuallz elasticnetpropensitymodel
#### but we can also compute them ourselves. 

### compute 1 propensity score per dataset for the R learner. we dont need the propensity score for the ols or for the testing. 

### BUT NOTICE THAT beacuse we dont feed propensitz score into the stacking step, we can see how mis-estimation 
### of the propensity score can skew the stacking step. Nie and Wager talk about how bias is absorbed into the intercept term. 

In [8]:
##############################################################################################
## R learner, three kinds of base regressors to estimate the treatment effect.
## three of these can be used to predict on the stacking set, and then create a stacking coefficient for each
##############################################################################################
# Calling the Base Learner class and feeding in XGB
learner_rXGB = BaseRRegressor(learner=XGBRegressor())
ate_r_XGBRegressor = learner_rXGB.estimate_ate(X=X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])
print('Using the BaseRRegressor class and using XGB:')
print(ate_r_XGBRegressor)

# Calling the Base Learner class and feeding in LinearRegression
## comes from from sklearn.linear_model import LinearRegression.. so i assume all can come from there???
learner_rLinearRegression = BaseRRegressor(learner=LinearRegression())
ate_r_LinearRegression = learner_rLinearRegression.estimate_ate(X=X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])
print('Using the BaseRRegressor class and using Linear Regression:')
print(ate_r_LinearRegression)

learner_decisionTree = BaseRRegressor(learner=DecisionTreeRegressor())
ate_r = learner_decisionTree.estimate_ate(X=X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])
print('Using the BaseRRegressor class and using DecisionTree:')
print(ate_r)

Using the BaseRRegressor class and using XGB:
(array([2.66232698]), array([2.65249818]), array([2.67215578]))
Using the BaseRRegressor class and using Linear Regression:
(array([2.28714406]), array([2.27948621]), array([2.29480191]))
Using the BaseRRegressor class and using DecisionTree:
(array([1.26162694]), array([1.18427609]), array([1.3389778]))


In [ ]:
## so now we have a 3 x 1000 array of predictions on the same dataset, from three different models.
## so we can only use some of our dataset to predict on, in order to train a model. 

## ok you can call fit_predict on all the data, but to fit the ols theres no sense in using all predictions, we can only use the ones we have a true treatment effect on.

cate_rXGB = learner_rXGB.fit_predict(X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])
cate_linear = learner_rLinearRegression.fit_predict(X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])
cate_decisionTree = learner_decisionTree.fit_predict(X['setup_A']['train'], treatment = assignment['setup_A']['train'], y=y['setup_A']['train'])


In [ ]:
## need to : 
# - take only the actually treated predictions from all predictions (the ones wiht a true treatment effect.)
# - store them in a dict ? or keep them in the array you know works for OLS. 

stacking_data = []
predictions = []

predictions.append(np.reshape(cate_rXGB, dim))
predictions.append(np.reshape(cate_linear, dim))
predictions.append(np.reshape(cate_decisionTree, dim))

for x in predictions:
    stacking_data.append(x[assignment[i][1] == 1])


stacking_data_dict = {}
predictions_dict = {}



#treated_elements = X[i][1][assignment[i][1] == 1]
#treated_y = y[i][1][assignment[i][1] == 1]

In [ ]:
### 
alpha=0.5
bins=30
plt.figure(figsize=(12,8))
plt.hist(stacking_data[i], alpha=alpha, bins=bins, label='R Learner - XGB')
plt.hist(treatment[i][1][assignment[i][1] == 1], alpha=alpha, bins=bins, label='true treatment effect')
plt.axvline(treatment[i][1][assignment[i][1] == 1].mean(), color='g', linestyle='dashed', linewidth=3)
plt.axvline(ate_r_XGBRegressor[1], color='r', linestyle='dashed', linewidth=2)

##(WHAT ABOUT THE TRUE ATE?)

plt.ylim(0, 40)
plt.title('predictions from xgb in the R learner, vs true treatment effect')
plt.xlabel('Individual Treatment Effect (CATE)')
plt.ylabel('# of Samples')
#_=plt.legend().subtract(in_num1, in_num2)

In [ ]:
A = treatment[i][1][assignment[i][1] == 1]
B = stacking_data[i]

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(A, B)

print(mse) 

### WHY IS MSE SO BAD HERE? 

In [ ]:
##### THE STACKING PART!!
## stacking data has three columns, of all the predictions on 332 values. 
## what is y output? for the stacking part it will be the true treatment effect: 

from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

y_stacking = treatment[i][1][assignment[i][1] == 1]
reg = LinearRegression().fit(np.transpose(stacking_data), y_stacking)

print(reg.coef_)
print(reg.intercept_)

### not a great sign for the coefs here is it?